In [3]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/ReconocimientoPatrones/genero.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df

,Person,height (feet),weight (lbs),foot size(inches)
0,male,6.00,180,12
1,male,5.92,190,11
2,male,5.58,170,12
3,male,5.92,165,10
4,female,5.00,100,6
5,female,5.50,150,8
6,female,5.42,130,7
7,female,5.75,150,9


In [5]:
# X contendra las caracteristicas
x = np.array(df.iloc[:,[1,2,3]])
x

array([[  6.  , 180.  ,  12.  ],
       [  5.92, 190.  ,  11.  ],
       [  5.58, 170.  ,  12.  ],
       [  5.92, 165.  ,  10.  ],
       [  5.  , 100.  ,   6.  ],
       [  5.5 , 150.  ,   8.  ],
       [  5.42, 130.  ,   7.  ],
       [  5.75, 150.  ,   9.  ]])

In [6]:
# y contendra las salidas de hombre y mujer expresadas en 1 y 0
y = list(map(lambda v:1 if v == 'male' else 0, df['Person']))
y

[1, 1, 1, 1, 0, 0, 0, 0]

In [7]:
import collections
y_dict = collections.Counter(y)
y_dict

Counter({1: 4, 0: 4})

In [8]:
pre_prob = np.zeros(2)
pre_prob

array([0., 0.])

In [9]:
pre_prob[0] = y_dict[0] / len(y)
pre_prob

array([0.5, 0. ])

In [10]:
def pre_probab(y):
  size = len(np.unique(y))
  y_dict = collections.Counter(y)
  # Crea un arreglo de ceros del mismo tamaño que el numero de clases
  # 2 en este caso (hombre y mujer)
  pre_prob = np.zeros(size)
  # itera en el numero de clases y coloca en cada posicion del arreglo pre-prob
  # el resultado de dividir el numero de hombres(o mujeres segun la iteracion) 
  # entre el total de personas
  for i in range(size):
    pre_prob[i] = y_dict[i] / len(y)
  return pre_prob

In [11]:
pre_prob = pre_probab(y)
pre_prob

array([0.5, 0.5])

In [12]:
x.shape

(8, 3)

In [13]:
n_features = x.shape[1]
n_features

3

In [14]:
mean = np.zeros((2, n_features))
var = np.zeros((2, n_features))
mean

array([[0., 0., 0.],
       [0., 0., 0.]])

In [15]:
var

array([[0., 0., 0.],
       [0., 0., 0.]])

In [16]:
# funcion de media y varianza por cada salida (en este caso: hombre y mujer)
def mean_var(x,y):
  # Numero de caracteristicas (3 en este caso: altura, peso, foot size)
  n_features = x.shape[1]
  # Numero de clases (2: hombre y mujer)
  size = size = len(np.unique(y))
  # Arreglo de ceros para la media
  m = np.zeros((size, n_features))
  # Arreglo de ceros para la varianza
  v = np.zeros((size, n_features))
  n_0 = np.bincount(y)[np.nonzero(np.bincount(y))[0]][0]
  # Aqui se almacenara el numero de elementos por caracteristica y por clase
  cases = np.ones((n_0, n_features))

  # Itera en nuestras dos salidas (t puede ser -> 0: Mujer, 1: Hombre)
  for t in np.unique(y):
    k = 0
    # Itera en los 8 elementos de nuestro dataset
    for i in range(x.shape[0]):
      # Si la posicion i de 'y' es igual a t
      if y[i] == t:
        # la posicion i de 'x' se añade a cases en la posicion k
        cases[k] = x[i]
        k += 1

    # Itera en las 3 caracteristicas
    for j in range(n_features):
      # Se calcula la media y varianza para cada caracteristica y para cada
      # salida (hombre y mujer)
      m[t][j] = np.mean(cases.T[j]) # calculo de la media
      v[t][j] = np.var(cases.T[j]) *(n_0/(n_0 - 1)) # calculo de la varianza

  return m, v # mean and variance 

In [17]:
mean, var = mean_var(x,y)
mean

array([[  5.4175, 132.5   ,   7.5   ],
       [  5.855 , 176.25  ,  11.25  ]])

In [18]:
var

array([[9.72250000e-02, 5.58333333e+02, 1.66666667e+00],
       [3.50333333e-02, 1.22916667e+02, 9.16666667e-01]])

In [19]:
# x1 es nuestra muestra a predecir si es hombre o mujer
x1 = x[1]
print(x1)

[  5.92 190.    11.  ]


In [20]:
# calculo de pre probabilidad para cada salida
pre_prob = pre_probab(y)
pre_prob

array([0.5, 0.5])

In [21]:
# calculo de media y varianza
mean, var = mean_var(x,y)

In [22]:
# Funcion que devuelve la probabilidad posterior
# Utiliza la media, la varianza y los valores de una muestra x
def prob_features_class(mean,var,x):
  n_features = mean.shape[1]
  size = len(np.unique(y))
  pfc = np.zeros(size)
  for i in range(0, size):
    Gauss = 1
    for j in range(0, n_features):
      Gauss = Gauss*(1/np.sqrt(2*np.pi*var[i,j]))* np.exp(-0.5*pow(
          (x[j] - mean[i][j]),2)/ var[i][j])
    
    pfc[i] = Gauss
  return pfc

In [23]:
# x1 sera la muestra a utilizar
pfc = prob_features_class(mean,var,x1)
pfc

array([2.39116430e-06, 1.34766867e-02])

In [24]:
# Calcula la probabilidad total sumando las probabilidades
# posterior y previas de cada una de las 2 salidas
total_prob = 0
for i in range(0,2):
  total_prob +=  pfc[i] * pre_prob[i]

In [25]:
total_prob

0.006739538942202236

In [26]:
# probabilidad condicional de cada una de las 2 salidas
pcf = np.zeros(2)
for i in range(0,2):
  pcf[i] = (pfc[i] * pre_prob[i])/total_prob

In [27]:
pcf

array([1.77398211e-04, 9.99822602e-01])

In [28]:
# Resultado de prediccion
# Toma el valor maximo de pcf
predict = int(pcf.argmax())
predict

1

In [29]:
# Funcion general, toma 'x', 'y', y una muestra x1
def GaussianNB(x,y, x1):
  y_dict = collections.Counter(y)
  size = len(y_dict.keys())
  pre_prob =  pre_probab(y)
  mean, var = mean_var(x,y)
  pfc = prob_features_class(mean,var,x1)
  pcf = np.zeros(size)
  total_prob = 0

  for i in range(0,size):
    total_prob +=  pfc[i] * pre_prob[i]

  for i in range(0,size):
    pcf[i] = (pfc[i] * pre_prob[i])/total_prob

  predict = int(pcf.argmax())

  return predict

In [30]:
# Test con x2
x2 = np.array([6, 130, 8])
print('Resultado de prediccion:', GaussianNB(x,y,x2))

Resultado de prediccion: 0


In [31]:
# Test con x5
x5 = np.array([[6,130, 8],[5.3,120, 6],[5.5,160, 7],[5.8,180, 11]])
for i in x5:
  result = GaussianNB(x,y, i)
  print('Prediction:',result)

Prediction: 0
Prediction: 0
Prediction: 0
Prediction: 1


In [32]:
# Test con x
accuracy = 0
n = -1
for i in x:
  n += 1
  result = GaussianNB(x,y, i)
  if result == y[n]:
    accuracy += 1
print('Accuracy:', accuracy/(n+1))

Accuracy: 1.0
